# Modelling a gallium arsenide surface

This example shows how to use the atomistic simulation environment or ASE for short,
to set up and run a particular calculation of a gallium arsenide surface.
ASE is a Python package to simplify the process of setting up,
running and analysing results from atomistic simulations across different simulation codes.
For more details on the integration DFTK provides with ASE,
see Atomistic simulation environment (ASE).

In this example we will consider modelling the (1, 1, 0) GaAs surface separated by vacuum.

Parameters of the calculation. Since this surface is far from easy to converge,
we made the problem simpler by choosing a smaller `Ecut` and smaller values
for `n_GaAs` and `n_vacuum`.
More interesting settings are `Ecut = 15` and `n_GaAs = n_vacuum = 20`.

In [1]:
miller = (1, 1, 0)   # Surface Miller indices
n_GaAs = 2           # Number of GaAs layers
n_vacuum = 4         # Number of vacuum layers
Ecut = 5             # Hartree
kgrid = (4, 4, 1);   # Monkhorst-Pack mesh

Use ASE to build the structure:

In [2]:
using ASEconvert
using PythonCall

a = 5.6537  # GaAs lattice parameter in Ångström (because ASE uses Å as length unit)
gaas = ase.build.bulk("GaAs", "zincblende"; a)
surface = ase.build.surface(gaas, miller, n_GaAs, 0, periodic=true);

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/pAQve/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/CondaPkg/0UqYV/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/83z4q/CondaPkg.toml
    CondaPkg Resolving changes
             + ase
             + libstdcxx
             + libstdcxx-ng
             + openssl
             + python
    CondaPkg Initialising pixi
             │ /home/runner/.julia/artifacts/cefba4912c2b400756d043a2563ef77a0088866b/bin/pixi
             │ init
             │ --format pixi
             └ /home/runner/work/DFTK.jl/DFTK.jl/docs/.CondaPkg
✔ Created /home/runner/work/DFTK.jl/DFTK.jl/docs/.CondaPkg/pixi.toml
    CondaPkg Wrote /home/runner/work/DFTK.jl/DFTK.jl/docs/.CondaPkg/pixi.toml
             │ [dependencies]
             │ openssl = ">=3, <3.6"
             │ libstdcxx = ">=3.4,<15.0"
             │ libstdcxx-ng = ">=3.4,<15.0"
             │ ase = ">=3.26,<3.27"
 

Get the amount of vacuum in Ångström we need to add

In [3]:
d_vacuum = maximum(maximum, surface.cell) / n_GaAs * n_vacuum
surface = ase.build.surface(gaas, miller, n_GaAs, d_vacuum, periodic=true);

Write an image of the surface and embed it as a nice illustration:

In [4]:
ase.io.write("surface.png", surface * pytuple((3, 3, 1)), rotation="-90x, 30y, -75z")

Python: None

<img src="https://docs.dftk.org/stable/surface.png" width=500 height=500 />

Use the `pyconvert` function from `PythonCall` to convert the ASE atoms
to an AtomsBase-compatible system.
This can then be used in the same way as other `AtomsBase` systems
(see AtomsBase integration for details) to construct a DFTK model:

In [5]:
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("cp2k.nc.sr.pbe.v0_1.largecore.gth")
model = model_DFT(pyconvert(AbstractSystem, surface);
                  functionals=PBE(),
                  temperature=1e-3,
                  smearing=DFTK.Smearing.Gaussian(),
                  pseudopotentials)

Model(gga_x_pbe+gga_c_pbe, 3D):
    lattice (in Bohr)    : [7.55469   , 0         , 0         ]
                           [0         , 7.55469   , 0         ]
                           [0         , 0         , 40.0648   ]
    unit cell volume     : 2286.6 Bohr³

    atoms                : As₂Ga₂
    pseudopot. family    : PseudoFamily("cp2k.nc.sr.pbe.v0_1.largecore.gth")

    num. electrons       : 16
    spin polarization    : none
    temperature          : 0.001 Ha
    smearing             : DFTK.Smearing.Gaussian()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(gga_x_pbe, gga_c_pbe)
                           Entropy()

In the above we use the `pseudopotential` keyword argument to
assign the respective pseudopotentials to the imported `model.atoms`.
Try lowering the SCF convergence tolerance (`tol`)
or try `mixing=KerkerMixing()` to see the full challenge of this system.

In [6]:
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis; tol=1e-6, mixing=LdosMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -16.58707202740                   -0.58    5.3    6.76s
  2   -16.72502331134       -0.86       -1.01    1.0    4.10s
  3   -16.73054923053       -2.26       -1.57    2.1    344ms
  4   -16.73121747872       -3.18       -2.16    1.0    232ms
  5   -16.73132366505       -3.97       -2.59    1.9    262ms
  6   -16.73133110399       -5.13       -2.83    2.0    269ms
  7   -16.73054127335   +   -3.10       -2.36    2.2    287ms
  8   -16.73132757574       -3.10       -2.98    2.3    266ms
  9   -16.73131253670   +   -4.82       -3.06    2.0    268ms
 10   -16.73131115197   +   -5.86       -3.05    1.9    245ms
 11   -16.73133480537       -4.63       -3.40    1.3    223ms
 12   -16.73133726263       -5.61       -3.52    1.0    216ms
 13   -16.73133884508       -5.80       -3.70    1.0    216ms
 14   -16.73134018640       -5.87       -4.50    1.2    220ms
 15   -1

In [7]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             5.8593969 
    AtomicLocal         -105.6100058
    AtomicNonlocal      2.3494808 
    Ewald               35.5044300
    PspCorrection       0.2016043 
    Hartree             49.5614205
    Xc                  -4.5976635
    Entropy             -0.0000035

    total               -16.731340200430